## SQL STREAMLIT

Este código en teoria nos va a servir para conectar nuestra base de datos en sql con streamlit

In [1]:
import streamlit as st
import mysql.connector
import pandas as pd

Configuramos una función para la conecsion y la obtención de datos de la base de datos

In [2]:
def connect_to_database():
    connection = mysql.connector.connect(
        host="localhost",       
        user="root",
        password="contraseña123",
        database="AIRBNB"
    )
    return connection


def fetch_data(query):
    connection = connect_to_database()
    cursor = connection.cursor()
    cursor.execute(query)
    result = cursor.fetchall()
    cursor.close()
    connection.close()
    return result

In [ ]:
# Interfaz de Streamlit
st.title("Dashboard de Airbnb using SQL")
# Opciones de visualización
opcion = st.selectbox("Select an analysis:", [
    "Ratings Distribution",
    "Number of Reviews vs Price",
    "Distribution by Property Type",
    "Correlation Map",
    "Prices by Property Type",
    "Average Price by Capacity",
    "Top 10 Most Offered Services"
])

In [ ]:
if opcion == "Ratings Distribution":
    query = """
        SELECT Ratings.ratings, COUNT(*) as count 
        FROM Ratings 
        GROUP BY Ratings.ratings 
        ORDER BY Ratings.ratings;
    """
    data = fetch_data(query)
    fig = px.bar(data, x="ratings", y="count", title="Ratings Distribution")
    st.plotly_chart(fig)

elif opcion == "Number of Reviews vs Price":
    query = """
        SELECT Ratings.num_reviews, Description_.prices_per_night 
        FROM Ratings 
        JOIN Description_ ON Ratings.record_id = Description_.id;
    """
    data = fetch_data(query)
    fig = px.scatter(data, x="num_reviews", y="prices_per_night", title="Number of Reviews vs Price")
    st.plotly_chart(fig)

elif opcion == "Distribution by Property Type":
    query = """
        SELECT Hosting.property_types, Ratings.ratings, Ratings.num_reviews, Description_.prices_per_night 
        FROM Hosting
        JOIN Ratings ON Hosting.record_id = Ratings.record_id
        JOIN Description_ ON Hosting.record_id = Description_.id;
    """
    data = fetch_data(query)
    fig = px.scatter(data, x="ratings", y="num_reviews", color="property_types",
                     size="prices_per_night", title="Distribution by Property Type")
    st.plotly_chart(fig)

elif opcion == "Correlation Map":
    query = """
        SELECT Ratings.ratings, Ratings.num_reviews, Description_.prices_per_night, Description_.maximum_guests 
        FROM Ratings
        JOIN Description_ ON Ratings.record_id = Description_.id;
    """
    data = fetch_data(query)
    correlation_matrix = data.corr()
    st.write("Correlation Map")
    st.dataframe(correlation_matrix)

elif opcion == "Prices by Property Type":
    query = """
        SELECT Hosting.property_types, Description_.prices_per_night 
        FROM Hosting
        JOIN Description_ ON Hosting.record_id = Description_.id;
    """
    data = fetch_data(query)
    fig = px.box(data, x="property_types", y="prices_per_night", title="Prices by Property Type")
    st.plotly_chart(fig)

elif opcion == "Average Price by Capacity":
    query = """
        SELECT Description_.maximum_guests, AVG(Description_.prices_per_night) as avg_price 
        FROM Description_ 
        GROUP BY Description_.maximum_guests;
    """
    data = fetch_data(query)
    fig = px.bar(data, x="maximum_guests", y="avg_price", title="Average Price by Capacity")
    st.plotly_chart(fig)

#elif opcion == "Top 10 Most Offered Services":
 #   query = """
  #      SELECT Services_.service, COUNT(*) as count 
   #     FROM Services_ 
    #    JOIN Services_Hosting ON Services_.service_id = Services_Hosting.service_id
     #   GROUP BY Services_.service 
      #  ORDER BY count DESC 
       # LIMIT 10;
    #"""
    #data = fetch_data(query)
    #fig = px.bar(data, x="service", y="count", title="Top 10 Most Offered Services")
    #st.plotly_chart(fig)
